In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
df_EU = pd.read_csv(f'../data/crunchbase-aggregated/EU1000.csv')

In [3]:
df_EU['Founders'].str.split(';')

0                                  [Viktor Chernomyrdin]
1                                                    [—]
2                                                    [—]
3                                                    [—]
4      [Claude Ritter,  Dave Bailey,  Kolja Hebenstre...
                             ...                        
995                                                  [—]
996                                       [Darren Ridge]
997                                                  [—]
998                                                  [—]
999                      [Juan Lobato,  Salvador Garcia]
Name: Founders, Length: 1000, dtype: object

In [4]:
df_EU['#founders'] = df_EU['Founders'].str.split(';').str.len()

In [5]:
for i in range(1000):
    if df_EU['Founders'].iloc[i] == '—':
        df_EU['#founders'].iloc[i] = 0

/Users/cinny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
# results_list = []
# for i in range(1000):
#     founders_list = df_EU['Founders'][i].split(';')
#     result_list = []
#     for name in founders_list:
#         name_block = name.split(' ')
#         first = ''.join(name_block[:-1])
#         last = name_block[-1]
#         result = !curl -X GET "https://v2.namsor.com/NamSorAPIv2/api2/json/gender/{first}/{last}" -H "accept: application/json" -H "X-API-KEY: fbeeb9ce36bee0728d3a577e10b06dc4"
#         result_list.append(result)
#     results_list.append(result_list)

In [7]:
# file = open('../data/api_results/EU1000_api.txt','w')
# file.writelines(str(results_list))
# file.close()

In [8]:
with open('../data/api_results/EU1000_api.txt', 'r') as f:
    api_result_string = f.read()

In [9]:
api_result_data = literal_eval(api_result_string)

In [10]:
literal_eval(api_result_data[0][0][5])['probabilityCalibrated']

0.9935642481532811

In [11]:
df_gender = pd.DataFrame({'api_raw': api_result_data})

In [12]:
gender_data = []
prob_data = [] # will need to check through prob<0.5
female_data = [] # number of females

# loop through rows
for i in range(1000):
    
    row = df_gender.iloc[i][0]
    
    gender_list = []
    prob_list = []
    
    count = 0
        
    # loop through name list
    for name in row:
        
        # yield results
        try: 
            gender = literal_eval(name[5])['likelyGender']
            gender_list.append(gender)

            prob = literal_eval(name[5])['probabilityCalibrated']
            prob_list.append(prob)

            if (gender == 'female') and (prob > 0.5):
                count += 1
        
        # if bad request, literal_eval can't read html on name[5]
        except SyntaxError:
            gender_list.append('N/A')
            prob_list.append('N/A')
            
    # founders col was entirely empty
    if gender_list == ['N/A']:
        count = 'N/A'
    
    gender_data.append(gender_list)
    prob_data.append(prob_list)
    female_data.append(count)

In [13]:
df_gender['gender'] = gender_data
df_gender['prob'] = prob_data
df_gender['#female'] = female_data

In [14]:
# df_gender

In [15]:
# check: good to know that all probs were higher than 50%

count = 0

for prob in prob_data:
    for p in prob:
        if type(p)==float and (p < 0.5):
            count += 1

count

0

In [16]:
df = pd.merge(left=df_EU, right=df_gender, left_index=True, right_index=True)

In [17]:
# df.to_csv(f'../data/crunchbase-aggregated/EU1000-gender.csv', index=False)